In [1]:
import TileCollection as TC
import TileGenerator as TG

import math
import random
import string
import subprocess
import os

In [6]:
#======================================================
# The function creates geojson file given parameters,
#   - place: geocode. (e.g. 'Beijing, China')
#   - buffer_size: radius centered place in kilometer
#   - tag: openstreet map feature tag
#   - values: openstreet map feature values
#   - file_name: file name saved as geojson
#======================================================

def featured_tiles(place, buffer_size, tag, values, file_name):
    boundaries = TC.boundaries(place = place, buffer_size = buffer_size)
    as_json = TC.map_features_json_response(bounds = boundaries, tag=tag, values=values)
    TC.json_to_geojson(as_json, file_name)
    return


#=======================================================================================
# The function creates shell scripts to collcec featured and non-featured tiles 
#  using geojson file.
#   - geojson_file_path: file path where geojson file exists
#   - zooms: zoom levels of tiles to be extracted 
#   - positive_file_name: shell script file name with path for positive dataset
#   - negative_file_name: shell script file name with path for negative dataset
#=======================================================================================
def sh_creator(geojson_file_path, zooms, positive_file_name, negative_file_name): 
    head = "#!/bin/bash"
    
    #-- write in .sh script    
    f = open(positive_file_name, 'w')
    print(head, file = f)
    
    f_non = open(negative_file_name, 'w')
    print(head, file = f_non)
    
    for zoom in zooms:   
        geojson_file_path = geojson_file_path
        gj = TG.load_json_file(geojson_file_path)
        features = TG.extract_feature_list_from_gj(gj)
        points_list = [f["geometry"]["coordinates"] for f in features]

        points = []
        urls = []
        for p in points_list:
            x, y = TG.deg2num(p[1],p[0],zoom)
            urls.append('https://' + random.choice('abc') + '.tile.openstreetmap.org/' + str(zoom) 
                        + '/' + str(x) + '/' + str(y) + '.png')

        urls_set = set(urls)
        
        x_val = []
        y_val = []
        counter = 0
        for u in urls_set:
            split_url = u.split('/')
            latlon = TG.num2deg(int(split_url[4]), int(split_url[5][:len(split_url[5])-4]), int(split_url[3]) )
            line_1 = 'wget -O ' + str(split_url[3]) + '_' + str(split_url[4]) + '_' + str(split_url[5]) + ' ' + u \
                      + ' #Lat: ' + str(latlon[0]) + ", Lon: " + str(latlon[1])
            print(line_1, file = f)
            if counter%50 == 0:
                line_2 = 'sleep 1.1'
                print(line_2, file = f)
            counter = counter + 1 
            
            z_val = zoom           #-- split_url[3]
            x_val.append(int(float(split_url[4])))
            y_val.append(int(float(split_url[5][:-4])))
            counter = counter + 1
            
        x_min, x_max, y_min, y_max = (min(x_val), max(x_val), min(y_val), max(y_val) )
        cnt = 0
        while cnt < counter:
            x_non = random.choice(list(range(x_min, x_max)))
            y_non = random.choice(list(range(y_min, y_max)))
            if not((x_non in x_val) & (y_non in y_val)):
                latlon = TG.num2deg(x_non, y_non, z_val)
                url_set = 'https://' + random.choice('abc') + '.tile.openstreetmap.org/' + str(z_val) + '/' + str(x_non) + '/' + str(y_non) + '.png'
                line_3 = 'wget -O ' + str(z_val) + '_' + str(x_non) + '_'+ str(y_non) + '.png' + ' ' + url_set + ' #Lat: ' + str(latlon[0]) + ", Lon: " + str(latlon[1])
                print(line_3, file = f_non)
                cnt += 1
            else:
                pass
            
            if cnt%50 == 0:
                line_4 = 'sleep 1.1'
                print(line_4, file = f_non)   
    return

    
#==========================================================================
# This function gives permission to created shell scripts
#  - sh_file_name: a string. List of .sh file names separated by space
#==========================================================================   
def shell_permission(sh_file_name):
    #os.chdir(file_dir)
    temp = subprocess.Popen(["chmod", "755", sh_file_name], stdout = subprocess.PIPE)
    data = subprocess.Popen(["ls", '-l', sh_file_name], stdout = subprocess.PIPE) 
    # subprocess.check_call([sh_file_name, "bash"])                
    return 

## 1. Get featured tiles as geojson
###   Example
* Extract tiles with 'aeroway' ('taxiway', 'runway','apron', 'helipad', 'heliport', 'terminal', 'aerodrome')
* Create "aeroways.geojson" file

In [4]:
featured_tiles(place="Beijing, China", buffer_size=500000, tag='military', 
               values=['airfield', 'bunker','barracks', 'checkpoint', 'danger_area', 'naval_base', 'nuclear_explosion_site'], 
               file_name='military_near_Beijing')

## 2. Using .geogson, create shell script for featured and non-featured classes

In [5]:
sh_creator(geojson_file_path='military_near_Beijing.geojson', zooms=[17,18], 
           positive_file_name = 'mil_Beijing_pos.sh', negative_file_name ='mil_Beijing_neg.sh')

## 3. Give permission to created shell scripts

In [8]:
shell_permission('mil_Beijing_pos.sh non-mil_Beijing_neg.sh')